# Position Predictor (In Progress)

`Given the position of 1,3,5 players + ball have the model try and predict the position of the other player.`

The intent of this notebook is to document the analysis of hypotheses related to the development of a position predictor model.  This analysis will be used in the construction and justification of a final model and pipeline.

## TODO

- [ ] refactor preprocessing to use tensorflow datasets
- [ ] add data expansion to preprocessing in a way that is sensible
- [ ] test state-of-art for MNIST trained on custom data

In [1]:
import carball
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import importlib
import itertools
import random
import xgboost as xgb
import tensorflow as tf
import tensorflow_datasets as tfds
import sklearn
import math
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.multioutput import MultiOutputRegressor

In [2]:
# CONFIGS
POSITION_COLUMNS = ['pos_x', 'pos_y', 'pos_z']
NCOLS = len(POSITION_COLUMNS)
RANDOM_SEED = 42
TEST_RATIO = 0.2
VALIDATE_RATIO = 0.2
PARTITION_RANGE = 1000
CACHE_PATH = "./.replay_cache"
RASTER_RESOLUTION = np.array((7,7,5))
RASTER_NCOLS = np.prod(RASTER_RESOLUTION)
PLAY_AREA_DIMS = np.array((4096*2, 5120*2, 2044))
PLAY_AREA_OFFSET = np.array((4096, 5120, 0))

In [3]:
random.seed(RANDOM_SEED)

## Data Retrieval (TODO)

This stage retrieves replays from the internet, and caches them locally.


### Train / Validation / Test Split (TODO)

Data partitioning should be done on a per-replay basis, so that there are no chances for the model to memorize similar frames from the same replay.

In [4]:
import data
importlib.reload(data)
data_manager = data.FileCachedCalculated()

Caching replays at: /home/axelmagn/.replays_cache


In [5]:
class Replay(object):
    def __init__(self, df, proto):
        self.df = df
        self.proto = proto

def get_replay(replay_id, manager):
    df = manager.get_pandas(replay_id)
    proto = manager.get_proto(replay_id)
    if df is None or proto is None:
        return None
    return Replay(df, proto)

def partition_replays(replay_list):
    return (
        list(filter(_is_train_partition, replay_list)),
        list(filter(_is_validate_partition, replay_list)),
        list(filter(_is_test_partition, replay_list)),
    )

_TRAIN_PARTITION = (1 - TEST_RATIO - VALIDATE_RATIO) * PARTITION_RANGE
_TEST_PARTITION = (1 - TEST_RATIO) * PARTITION_RANGE

def _is_train_partition(replay_id):
    id_value = abs(hash(replay_id)) % PARTITION_RANGE
    return id_value < _TRAIN_PARTITION

def _is_validate_partition(replay_id):
    id_value = abs(hash(replay_id)) % PARTITION_RANGE
    return id_value >= _TRAIN_PARTITION and id_value < _TEST_PARTITION

def _is_test_partition(replay_id):
    id_value = abs(hash(replay_id)) % PARTITION_RANGE
    return id_value >= _TEST_PARTITION

In [6]:
# for now, we will just do a manual split of the first 10 replays
replays = data_manager.get_replay_list(num=50)
train_replays, validate_replays, test_replays = partition_replays(replays)
replays[:5]

['D4DE3A894229D5B8A5D0AE9091D3CA6C',
 '4DC852DA4D28F5D9047C509CA03412C8',
 'EF578C404F791CE044EFDEB0036DA5EA',
 'B828B7FD472EE7A19E9A9C8ECB7CA14B',
 'A21C39DD402315E9831EC58C7A331F64']

In [7]:
len(train_replays), len(validate_replays), len(test_replays)

(29, 11, 10)

## Preprocessing

This stage takes replay dataframes and protobufs and prepares them as a numpy matrix suitable for model ingestion and analysis. There are 2 possible schemas I want to explore for state representation. 


### Schema: analog positions (DONE)

Records shall be represented as position vectors as shown below

```
<ball> <t0_p0> <t0_p1> <t0_p2> <t1_p0> <t1_p1> <t1_p2>
foreach: <pos_x> <pos_y> <pos_z>
```

### Schema: rasterized positions (TODO)

Records shall be represented as 3-channel rasterized images, with a channel each for ball, team_0, and team_1.   This can be generated from the condensed positions.

### Filtering (TODO)

Starting positions are likely to be over-represented in this dataset.  It may be useful to filter out all frames before a ball has been hit.

### Reordering (TODO)

The data could then be expanded to capture the symmetries of team and player reordering.  Player ordering in the condensed schema is arbitrary, and each record could generate `3! * 3! = 36` permutations. Team ordering is not arbitrary, but teams could be swapped by negating `pos_y` values.  This would increase the total available permutations to `36 * 2 = 72`.  There is also x-axis symmetry, so by negating X values we get another mirror, bringing it to `72 * 2 = 144` permutations.

### Distortion (TODO)

Optionally, it would be possible to slightly randomize positions, in order to help generalize.


### Dropout (TODO)

The original plan was to omit one player, and make the predictor guess it. If we are already have generated all reorderings, it should be sufficient to simply omit the final player.  However, I would also like to explore the possibility of a GAN or Autoencoder where one position within a valid record is fabricated, and the predictor must reconstruct the valid position from the invalid one.

In [8]:
def analog_positions(replay):
    assert replay is not None
    proto = replay.proto
    dataframe = replay.df
    assert proto is not None
    assert dataframe is not None
    ball_posn = get_named_posn(dataframe, 'ball')
    ball_posn = add_col_prefix(ball_posn, "ball_")
    t0_posns = []
    t1_posns = []
    for player in proto.players:
        posn = get_named_posn(dataframe, player.name)
        if player.is_orange == 0:
            t0_posns.append(posn)
        else:
            t1_posns.append(posn)
    for i, posn in enumerate(t0_posns):
        t0_posns[i] = add_col_prefix(posn, 't0_p{}_'.format(i))
    for i, posn in enumerate(t1_posns):
        t1_posns[i] = add_col_prefix(posn, 't1_p{}_'.format(i))
    positions = [ball_posn] + t0_posns + t1_posns
    return pd.concat(positions, axis=1).dropna()

def get_named_posn(dataframe, name):
    return dataframe[name][POSITION_COLUMNS]

def get_named_player(proto, player_name):
    for player in proto.players:
        if player.name == player_name:
            return player
    raise ValueError("Could not find player: " + player_name)
    
def add_col_prefix(dataframe, prefix):
    return dataframe.rename(lambda col: prefix + col, axis='columns')

In [9]:
loaded_replays = filter(lambda r: r is not None, (get_replay(replay_name, data_manager) for replay_name in train_replays))
train_aps = pd.concat((analog_positions(r) for r in loaded_replays), axis=0)
train_aps.head()

E0603 21:00:02.017177 139964721129088 pandas_manager.py:29] Failure to read pandas [] from memory: Struct error
Traceback (most recent call last):
  File "/home/axelmagn/workspace/personal/ReplayModels/venv/lib/python3.7/site-packages/carball/analysis/utils/numpy_manager.py", line 49, in get_array
    starting_byte = struct.unpack('i', chunk)[0]
struct.error: unpack requires a buffer of 4 bytes

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/axelmagn/workspace/personal/ReplayModels/venv/lib/python3.7/site-packages/carball/analysis/utils/pandas_manager.py", line 27, in safe_read_pandas_to_memory
    return PandasManager.read_numpy_from_memory(buffer)
  File "/home/axelmagn/workspace/personal/ReplayModels/venv/lib/python3.7/site-packages/carball/analysis/utils/pandas_manager.py", line 40, in read_numpy_from_memory
    array = read_array_from_file(buffer)
  File "/home/axelmagn/workspace/personal/ReplayModels/venv/lib/

,ball_pos_x,ball_pos_y,ball_pos_z,t0_p0_pos_x,t0_p0_pos_y,t0_p0_pos_z,t0_p1_pos_x,t0_p1_pos_y,t0_p1_pos_z,t0_p2_pos_x,...,t0_p2_pos_z,t1_p0_pos_x,t1_p0_pos_y,t1_p0_pos_z,t1_p1_pos_x,t1_p1_pos_y,t1_p1_pos_z,t1_p2_pos_x,t1_p2_pos_y,t1_p2_pos_z
index,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,93.0,1952.0,-2464.0,18.0,-256.0,-3840.0,18.0,256.0,...,18.0,-1951.0,2465.0,18.0,257.0,3840.0,18.0,-255.0,3840.0,18.0
1,0.0,0.0,93.0,1952.0,-2464.0,18.0,-256.0,-3840.0,18.0,256.0,...,18.0,-1951.0,2465.0,18.0,257.0,3840.0,18.0,-255.0,3840.0,18.0
2,0.0,0.0,93.0,1952.0,-2464.0,18.0,-256.0,-3840.0,18.0,256.0,...,18.0,-1951.0,2465.0,18.0,257.0,3840.0,18.0,-255.0,3840.0,18.0
3,0.0,0.0,93.0,1952.0,-2464.0,18.0,-256.0,-3840.0,18.0,256.0,...,18.0,-1951.0,2465.0,18.0,257.0,3840.0,18.0,-255.0,3840.0,18.0
4,0.0,0.0,93.0,1952.0,-2464.0,18.0,-256.0,-3840.0,18.0,256.0,...,18.0,-1951.0,2465.0,18.0,257.0,3840.0,18.0,-255.0,3840.0,18.0


In [10]:
loaded_replays = filter(lambda r: r is not None, (get_replay(replay_name, data_manager) for replay_name in validate_replays))
validate_aps = pd.concat((analog_positions(r) for r in loaded_replays), axis=0)

E0603 21:00:07.348217 139964721129088 pandas_manager.py:29] Failure to read pandas [] from memory: Struct error
Traceback (most recent call last):
  File "/home/axelmagn/workspace/personal/ReplayModels/venv/lib/python3.7/site-packages/carball/analysis/utils/numpy_manager.py", line 49, in get_array
    starting_byte = struct.unpack('i', chunk)[0]
struct.error: unpack requires a buffer of 4 bytes

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/axelmagn/workspace/personal/ReplayModels/venv/lib/python3.7/site-packages/carball/analysis/utils/pandas_manager.py", line 27, in safe_read_pandas_to_memory
    return PandasManager.read_numpy_from_memory(buffer)
  File "/home/axelmagn/workspace/personal/ReplayModels/venv/lib/python3.7/site-packages/carball/analysis/utils/pandas_manager.py", line 40, in read_numpy_from_memory
    array = read_array_from_file(buffer)
  File "/home/axelmagn/workspace/personal/ReplayModels/venv/lib/

In [11]:
# Split features and targets
def get_analog_features(analog_positions_dataframe):
    return np.copy(analog_positions_dataframe.values[:,:-NCOLS])

def get_analog_targets(analog_positions_dataframe):
    return np.copy(analog_positions_dataframe.values[:,-NCOLS:])

# TODO: split train, validation, and test sets based on multiple replays

In [12]:
train_aps_features = get_analog_features(train_aps)
train_aps_targets = get_analog_targets(train_aps)
validate_aps_features = get_analog_features(validate_aps)
validate_aps_targets = get_analog_targets(validate_aps)
train_aps_features.shape

(234458, 18)

In [13]:
# rasterization.  

def rasterize_row(aps):
    """
    convert xyz coordinates for a single row into an image where pixels reflect number 
    of entities in that pixel's cell, normalized between 0 and 1.
    
    input schema: (position_columns,)
    output schema (raster_x, raster_y, raster_z)
    """
    n_entities = math.floor(aps.shape[0] / NCOLS)
    aps = aps.reshape((n_entities, NCOLS))
    aps = np.add(aps, PLAY_AREA_OFFSET)
    aps = np.divide(aps, PLAY_AREA_DIMS)
    aps = np.multiply(aps, RASTER_RESOLUTION)
    aps = np.apply_along_axis(rasterize_pos, 1, aps)
    aps = np.sum(aps, axis=0)
    return aps
  
def rasterize_pos(position):
    """
    convert a single xyz position into a rasterized image
    """
    out = np.zeros(RASTER_RESOLUTION)
    position = np.floor(position)
    (x, y, z) = map(int, tuple(position))
    out[x,y,z] = 1
    return out

def rasterize_batch(aps):
    return np.apply_along_axis(rasterize_row, 1, aps)

In [16]:
# train_rps_features = rasterize_batch(train_aps_features)
a1 = np.array([[1,2,3,4,5,6],
           [9,8,7,6,5,4]])
rasterize_batch(train_aps_features)

IndexError: index 7 is out of bounds for axis 1 with size 7

## Model Selection

This stage evaluates different models on the training data.  Below is a list of model hypotheses that should be considered.

### Models

#### XGBoost (Baseline) (In Progress)

XGBoost performs well at many complex regression problems, and is easy to apply using a library.  It will serve as the baseline against which more sophisticated model hypotheses can be compared.

#### Clustering / KNN (Baseline) (TODO)

Another simple solution would be to retrieve similar memorized positions.

#### Deep Neural Networks (TODO)

After baselines have been measured, more sophisticated models such as convolutional neural networks can be considered.

### Selection Criteria

As important as the models is our criteria for evaluating them.  Since this problem does not easily map to a real-world loss function, this can be somewhat arbitrary, and may depend on the output schema.

#### RMSE (Single Output Schema)

Old reliable.

#### MAE (Single Output Schema)

It may be justifiable to use this since I don't care too much about missing outliers.

#### R^2 (Single Output Schema)

SKLearn default.  Seems to have some advantages.  Will probably stick with this out of convenience.

#### Probability Weighted Score (Distribution Output Schema)

If the model outputs a probability distribution across multiple possible positions, I think it makes sense to add up the score of those positions, multiplied by their predicted probability weight.



## XGBoost

Try training with XGBoost.  Key questions:

1. would we benefit from more training data?
2. would we benefit from more iterations?
3. Are the hyperparameters optimaly tuned?

In [25]:
def xgb_model(**kwargs):
    return MultiOutputRegressor(xgb.XGBRegressor(learning_rate=0.01, objective='reg:squarederror', random_seed=0))

In [29]:
model = xgb_model()
model.fit(train_features, train_targets)

MultiOutputRegressor(estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1, gamma=0,
                                            importance_type='gain',
                                            learning_rate=0.01,
                                            max_delta_step=0, max_depth=3,
                                            min_child_weight=1, missing=None,
                                            n_estimators=100, n_jobs=1,
                                            nthread=None,
                                            objective='reg:squarederror',
                                            random_seed=0, random_state=0,
                                            reg_alpha=0, reg_lambda=1,
                                            scale_pos_weight=1, seed=None,
          

In [31]:
# how does this model perform out of the box?
model.score(validate_features, validate_targets)

0.23961912393954052

In [32]:
# TODO: give a shit about the other questions

## Simple Dense Classifier

## TMP: Tensorflow sanity check

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

11493376/11490434 [==============================] - 3s 0us/step


In [3]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [4]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [6]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [7]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [8]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [11]:

EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))


Epoch 1, Loss: 0.1341739147901535, Accuracy: 95.8933334350586, Test Loss: 0.05695280432701111, Test Accuracy: 98.08999633789062
Epoch 2, Loss: 0.0873652845621109, Accuracy: 97.33583068847656, Test Loss: 0.05193914473056793, Test Accuracy: 98.22000122070312
Epoch 3, Loss: 0.06430377066135406, Accuracy: 98.03722381591797, Test Loss: 0.05261257290840149, Test Accuracy: 98.26333618164062
Epoch 4, Loss: 0.05145478621125221, Accuracy: 98.4254150390625, Test Loss: 0.053377918899059296, Test Accuracy: 98.32250213623047
Epoch 5, Loss: 0.042890507727861404, Accuracy: 98.68299865722656, Test Loss: 0.054339755326509476, Test Accuracy: 98.35399627685547


In [10]:
tf.test.is_gpu_available()

False